In [8]:
import pandas as pd
import numpy as np
import gc
from sklearn.metrics import roc_auc_score
from collections import defaultdict
from tqdm.notebook import tqdm
import lightgbm as lgb
from datatable import dt, fread
from utils.feature_eng import *

In [18]:
train_path = '/home/carlchao/Riiid-/CV/small_csv_file/cv1_train.csv'
valid_path = '/home/carlchao/Riiid-/CV/small_csv_file/cv1_valid.csv'
question_file = '/home/carlchao/Riiid_data/data/questions.csv'

In [3]:

# def add_user_feats(df, answered_correctly_sum_u_dict, count_u_dict, content_dict):
#     acsu = np.zeros(len(df), dtype=np.int32)
#     cu = np.zeros(len(df), dtype=np.int32)
#     bool = np.zeros(len(df), dtype=np.int8)
    
#     for cnt,row in enumerate(tqdm(df[['user_id','answered_correctly','content_id']].values)):
#         acsu[cnt] = answered_correctly_sum_u_dict[row[0]]
#         cu[cnt] = count_u_dict[row[0]]
#         answered_correctly_sum_u_dict[row[0]] += row[1]
#         count_u_dict[row[0]] += 1
        
#         if row[2] not in content_dict[row[0]]:
#             content_dict[row[0]].append(row[2])
#             bool[cnt] = 1
#         else:
#             bool[cnt] = 0
            
#     user_feats_df = pd.DataFrame({'answered_correctly_sum_u':acsu, 'count_u':cu, 'first_time': bool})
#     user_feats_df['answered_correctly_avg_u'] = user_feats_df['answered_correctly_sum_u'] / user_feats_df['count_u']
#     df = pd.concat([df, user_feats_df], axis=1)
    
#     return df

# def add_user_feats_without_update(df, answered_correctly_sum_u_dict, count_u_dict, content_dict):
#     acsu = np.zeros(len(df), dtype=np.int32)
#     cu = np.zeros(len(df), dtype=np.int32)
#     bool = np.zeros(len(df), dtype=np.int8)
    
#     for cnt,row in enumerate(df[['user_id','content_type_id']].values):
#         acsu[cnt] = answered_correctly_sum_u_dict[row[0]]
#         cu[cnt] = count_u_dict[row[0]]
        
#         if row[1] not in content_dict[row[0]]:
#             bool[cnt] = 1
#         else:
#             bool[cnt] = 0
            
#     user_feats_df = pd.DataFrame({'answered_correctly_sum_u':acsu, 'count_u':cu, 'first_time': bool})
#     user_feats_df['answered_correctly_avg_u'] = user_feats_df['answered_correctly_sum_u'] / user_feats_df['count_u']
#     df = pd.concat([df, user_feats_df], axis=1)
#     return df

# def update_user_feats(df, answered_correctly_sum_u_dict, count_u_dict, content_dict):
#     for row in df[['user_id','answered_correctly','content_type_id']].values:
#         if row[2] == 0:
#             answered_correctly_sum_u_dict[row[0]] += row[1]
#             count_u_dict[row[0]] += 1
#             content_dict[row[0]].append(row[2])

In [10]:


def add_user_feats(df, answered_correctly_sum_u_dict, count_u_dict, content_dict):
    acsu = np.zeros(len(df), dtype=np.int32)
    cu = np.zeros(len(df), dtype=np.int32)
    bool = np.zeros(len(df), dtype=np.int8)
    
    for cnt,row in enumerate(tqdm(df[['user_id','answered_correctly','content_id']].values)):
        acsu[cnt] = answered_correctly_sum_u_dict[row[0]]
        cu[cnt] = count_u_dict[row[0]]
        answered_correctly_sum_u_dict[row[0]] += row[1]
        count_u_dict[row[0]] += 1
        
        if row[2] not in content_dict[row[0]]:
            content_dict[row[0]].append(row[2])
            bool[cnt] = 1
        else:
            bool[cnt] = 0
            
    user_feats_df = pd.DataFrame({'answered_correctly_sum_u':acsu, 'count_u':cu, 'first_time': bool})
    user_feats_df['answered_correctly_avg_u'] = user_feats_df['answered_correctly_sum_u'] / user_feats_df['count_u']
    df = pd.concat([df, user_feats_df], axis=1)
    
    return df

def add_user_feats_without_update(df, answered_correctly_sum_u_dict, count_u_dict, content_dict):
    acsu = np.zeros(len(df), dtype=np.int32)
    cu = np.zeros(len(df), dtype=np.int32)
    bool = np.zeros(len(df), dtype=np.int8)
    
    for cnt,row in enumerate(df[['user_id','content_type_id']].values):
        acsu[cnt] = answered_correctly_sum_u_dict[row[0]]
        cu[cnt] = count_u_dict[row[0]]
        
        if row[1] not in content_dict[row[0]]:
            bool[cnt] = 1
        else:
            bool[cnt] = 0
            
    user_feats_df = pd.DataFrame({'answered_correctly_sum_u':acsu, 'count_u':cu, 'first_time': bool})
    user_feats_df['answered_correctly_avg_u'] = user_feats_df['answered_correctly_sum_u'] / user_feats_df['count_u']
    df = pd.concat([df, user_feats_df], axis=1)
    return df

def update_user_feats(df, answered_correctly_sum_u_dict, count_u_dict, content_dict):
    for row in df[['user_id','answered_correctly','content_type_id']].values:
        if row[2] == 0:
            answered_correctly_sum_u_dict[row[0]] += row[1]
            count_u_dict[row[0]] += 1
            content_dict[row[0]].append(row[2])

In [11]:
train = pd.read_pickle('preprocess_fea/small_train.pickle')
valid = pd.read_pickle('preprocess_fea/small_valid.pickle')

In [ ]:
import pickle
with open("preprocess_fea/small_answered_correctly_sum_u_dict.pickle", "rb") as filename:  
    a =  pickle.load(filename)
    filename.close()
    
with open("preprocess_fea/small_count_u_dict.pickle", "rb") as filename:  
    b = pickle.load(filename)
    filename.close()
    
with open("preprocess_fea/small_content_dict.pickle", "rb") as filename:  
    c = pickle.load(filename)
    filename.close()


In [5]:
# fill with mean value for prior_question_elapsed_time
# note that `train.prior_question_elapsed_time.mean()` dose not work!
# please refer https://www.kaggle.com/its7171/can-we-trust-pandas-mean for detail.
# prior_question_elapsed_time_mean = train.prior_question_elapsed_time.dropna().values.mean()
# train['prior_question_elapsed_time_mean'] = train.prior_question_elapsed_time.fillna(prior_question_elapsed_time_mean)
# valid['prior_question_elapsed_time_mean'] = valid.prior_question_elapsed_time.fillna(prior_question_elapsed_time_mean)

# use only last 30M training data for limited memory on kaggle env.
#train = train[-30000000:]

# part
questions_df = pd.read_csv(question_file)
en = Tags_encoder(3)
questions_df = pd.concat([questions_df, en.fit_transform(questions_df['tags'].fillna(' '))], 1)

train = pd.merge(train, questions_df[['question_id', 'part','PCA_0','PCA_3','PCA_4']], left_on = 'content_id', right_on = 'question_id', how = 'left')
valid = pd.merge(valid, questions_df[['question_id', 'part','PCA_0','PCA_3','PCA_4']], left_on = 'content_id', right_on = 'question_id', how = 'left')


In [6]:
# tmp = train.groupby([['user_id','part','answered_correctly']])['answered_correctly'].agg(['mean','sum']).reset_index()
# tmp.columns = ['auc_part_u', 'sum_part_u']

,row_id,timestamp,user_id,content_id,content_type_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,answered_correctly_avg_c,answered_correctly_sum_u,count_u,first_time,answered_correctly_avg_u,question_id,part,PCA_0,PCA_1,PCA_2
0,32933156,0,705741139,128,False,1,NaN,True,0.966887,0,0,1,NaN,128,1,-0.072154,-0.012893,-0.022862
1,32933157,20666,705741139,7860,False,1,16000.0,False,0.954720,1,1,1,1.0,7860,1,-0.079403,-0.017725,-0.015423
2,32933158,39172,705741139,7922,False,1,19000.0,False,0.953161,2,2,1,1.0,7922,1,-0.072154,-0.012893,-0.022862
3,32933159,58207,705741139,156,False,1,17000.0,False,0.932764,3,3,1,1.0,156,1,-0.070269,-0.003667,-0.037742
4,32933160,75779,705741139,51,False,1,17000.0,False,0.930161,4,4,1,1.0,51,1,-0.071638,-0.014265,-0.017627


In [ ]:
#drop useless col
train.drop(['timestamp','user_id','content_id','content_type_id','question_id'], axis=1, inplace=True)
valid.drop(['timestamp','user_id','content_id','content_type_id','question_id'], axis=1, inplace=True)

# changing dtype to avoid lightgbm error
train['prior_question_had_explanation'] = train.prior_question_had_explanation.fillna(False).astype('int8')
valid['prior_question_had_explanation'] = valid.prior_question_had_explanation.fillna(False).astype('int8')

In [ ]:
TARGET = 'answered_correctly'
FEATS = ['answered_correctly_avg_u', 'answered_correctly_sum_u', 'count_u', 'answered_correctly_avg_c', 'part', 'prior_question_had_explanation', 'prior_question_elapsed_time','first_time']
FEATS = FEATS + ['PCA_0','PCA_3','PCA_4']
dro_cols = list(set(train.columns) - set(FEATS))
y_tr = train[TARGET]
y_va = valid[TARGET]
train.drop(dro_cols, axis=1, inplace=True)
valid.drop(dro_cols, axis=1, inplace=True)
_=gc.collect()

In [ ]:
lgb_train = lgb.Dataset(train[FEATS], y_tr, categorical_feature=['part', 'prior_question_had_explanation'])
lgb_valid = lgb.Dataset(valid[FEATS], y_va, categorical_feature=['part', 'prior_question_had_explanation'])
del train, y_tr
_=gc.collect()

In [ ]:
model = lgb.train(
                    {'objective': 'binary'}, 
                    lgb_train,
                    valid_sets=[lgb_train, lgb_valid],
                    verbose_eval=100,
                    num_boost_round=10000,
                    early_stopping_rounds= 400,
                    categorical_feature=['part', 'prior_question_had_explanation']
    
                )
print('auc:', roc_auc_score(y_va, model.predict(valid[FEATS])))
_ = lgb.plot_importance(model)

In [ ]:
# tags useful 
# user for each part auc substract all user each part auc
# to do join lectures tags ==> question tags, lecture tags encode
# content_id useful

# optuna


In [ ]:
#baseline: 75.84

## add tags6: 

##pca
# pca[0,1,2,3,4] : 76.01
# pca[0,1,2]: 75.99
# pca[0,3,4]: 76.00
# pca[0,3,4,5,6]: 76.00

## pca[0,3,4] + part feature: 76.34
